In [1]:
# Block 1: Install necessary libraries
!pip install scikit-learn pandas matplotlib gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00


In [2]:
# Block 2: Import libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import gradio as gr
import matplotlib.pyplot as plt
import io
import base64

In [10]:
def customer_clustering(data_csv, n_clusters):
    """
    Performs customer profile clustering using K-means and generates a visualization.

    Args:
        data_csv (file): Uploaded CSV file containing customer data.
        n_clusters (int): Number of clusters.

    Returns:
        str: Base64 encoded image of the cluster plot.
    """

    try:
        df = pd.read_csv(data_csv.name)
        numerical_features = df.select_dtypes(include=np.number).columns.tolist()

        if numerical_features:
            numerical_features = numerical_features[1:]

        if not numerical_features:
            return "Error: No valid numerical features found in the CSV (excluding ID)."

        # Explicitly convert to numeric (error handling)
        for col in numerical_features:
            try:
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                return f"Error: Column '{col}' contains non-numeric data."

        print("Numerical Features:", numerical_features) # Debugging line
        print("Dataframe head: ", df.head()) # Debugging line
        print("Dataframe dtypes: ", df.dtypes) # Debugging line

        X = df[numerical_features]
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
        df['cluster'] = kmeans.fit_predict(X_scaled)

        if len(numerical_features) >= 2:
            plt.figure(figsize=(10, 6))
            plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=df['cluster'], cmap='viridis')
            plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red', marker='X')
            plt.title('Customer Clusters')
            plt.xlabel(numerical_features[0] + " (Scaled)")
            plt.ylabel(numerical_features[1] + " (Scaled)")
            plt.colorbar(label='Cluster')

            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            img_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
            plt.close()

            return f"data:image/png;base64,{img_base64}"

        else:
            return "Error: need at least 2 numerical columns for plotting (excluding ID)."

    except Exception as e:
        return f"An error occurred: {e}"

In [12]:
# Block 4: Create the Gradio interface
iface = gr.Interface(
    fn=customer_clustering,
    inputs=[
        gr.File(label="Upload Customer Data (CSV)"),
        gr.Slider(minimum=2, maximum=10, step=1, label="Number of Clusters"),
    ],
    outputs=gr.Image(type="pil", label="Cluster Visualization"),
    title="Customer Profile Clustering",
    description="Upload a CSV file with customer data and specify the number of clusters to visualize customer segments."
)

In [11]:
# Block 5: Launch the Gradio interface
iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc343e6c9e38c877d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
